In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from keras_tuner import RandomSearch
from keras.callbacks import EarlyStopping
import random

# Define the file paths for each period
file_paths = {
    'bear_market_1': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_1_regression.csv',
    'bear_market_2': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bear_market_2_regression.csv',
    'bull_market_1': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_1_regression.csv',
    'bull_market_2': '/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/bull_market_2_regression.csv'
}

# Initialize the results dictionary
results = {}

# Seed value for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
columns_to_scale = ['Open', 'High', 'Low', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

scaler = MinMaxScaler(feature_range=(0, 1))

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3
close_index = 3

# Define a model-building function
def build_model(hp):
    model = Sequential()
    filters = hp.Int('filters', min_value=10, max_value=150, step=5)
    pool = hp.Choice('pool_size', values=[2, 3])
    model.add(Conv2D(filters=filters,
                    kernel_size=3, activation='relu', padding='same', input_shape=(window_size, len(columns_to_scale), 1)))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))
    
    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))

    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool,padding='same'))
    model.add(Flatten())
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error',
                  metrics=['mae', 'mean_absolute_percentage_error'])
    return model

# Function to fit the model and evaluate metrics
def fit_and_evaluate(data, scaler, window_size, close_index, project_name):
    data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])
    training_size = int(len(data) * 0.9)
    training_data = data[:training_size]
    test_data = data[training_size:]
    train_data = training_data[:int(len(training_data) * 0.9)]
    val_data = training_data[int(len(training_data) * 0.9):]

    X_train, y_train = create_dataset(train_data, window_size, close_index)
    X_test, y_test = create_dataset(test_data, window_size, close_index)
    X_val, y_val = create_dataset(val_data, window_size, close_index)
    X_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

    X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2], 1))
    X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))
    X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2], 1))
    X_train_full = X_train_full.reshape((X_train_full.shape[0], window_size, X_train_full.shape[2], 1))

    tuner = RandomSearch(
        build_model,
        objective='val_mean_absolute_percentage_error',
        max_trials=200,
        executions_per_trial=1,
        directory='my_dir',
        project_name=project_name,
        overwrite=False
    )

    early_stopping = EarlyStopping(monitor='val_mean_absolute_percentage_error', patience=15, restore_best_weights=True)

    tuner.search(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=2, callbacks=[early_stopping])

    best_hps = tuner.get_best_hyperparameters()[0]

    mae_list, rmse_list, mape_list = [], [], []
    for _ in range(10):
        best_model = tuner.hypermodel.build(best_hps)  
        history = best_model.fit(X_train_full, y_train_full, epochs=250, batch_size=32, verbose=1)
        predictions = best_model.predict(X_test)

        mae = mean_absolute_error(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        mape = tf.keras.losses.MAPE(y_test, predictions)

        mae_list.append(mae)
        rmse_list.append(rmse)
        mape_list.append(np.mean(mape))

    return {
        'mae': np.mean(mae_list),
        'rmse': np.mean(rmse_list),
        'mape': np.mean(mape_list)
    }

# Process each period and store results
for period, file_path in file_paths.items():
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    project_name = f'CNN-3D-REG-{period}'
    results[period] = fit_and_evaluate(data, scaler, window_size, close_index, project_name)

# Create a DataFrame from the results
results_df = pd.DataFrame(results).T
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('CNN_BL_BR_performance_summary_regression.csv')


Reloading Tuner from my_dir/CNN-3D-REG-bear_market_1/tuner0.json
Epoch 1/250


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 54290188.0000 - mae: 6758.7812 - mean_absolute_percentage_error: 81.0032  
Epoch 2/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7570467.5000 - mae: 2416.2180 - mean_absolute_percentage_error: 28.8093
Epoch 3/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 2449626.2500 - mae: 1329.5194 - mean_absolute_percentage_error: 15.6569
Epoch 4/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 750614.3125 - mae: 661.8164 - mean_absolute_percentage_error: 7.7598
Epoch 5/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 524229.0000 - mae: 472.5970 - mean_absolute_percentage_error: 5.3190
Epoch 6/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 441327.0000 - mae: 441.2931 - mean_absolute_percentage_error: 4.8249
Epoch 7/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 404809.6250 - mae: 438.2346 - mean_absolute_percentage_error: 4.8157
Epoch 8/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 373645.7812 - mae: 385.1779 - mean_absolute_p

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 35006080.0000 - mae: 4838.5864 - mean_absolute_percentage_error: 57.9082  
Epoch 2/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1461098.6250 - mae: 919.9699 - mean_absolute_percentage_error: 10.6325
Epoch 3/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 771448.6250 - mae: 629.3264 - mean_absolute_percentage_error: 7.1133
Epoch 4/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 522562.7500 - mae: 483.0886 - mean_absolute_percentage_error: 5.3084
Epoch 5/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 482071.2812 - mae: 490.1907 - mean_absolute_percentage_error: 5.4328
Epoch 6/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 532324.3125 - mae: 542.1932 - mean_absolute_percentage_error: 6.1565
Epoch 7/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 510002.5938 - mae: 544.2751 - mean_absolute_percentage_error: 6.3097
Epoch 8/250
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 399651.2500 - mae: 458.0396 - mean_absolute_perc

...

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 131579752.0000 - mae: 6784.3882 - mean_absolute_percentage_error: 42.4824   
Epoch 2/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2220984.5000 - mae: 827.5753 - mean_absolute_percentage_error: 4.6197
Epoch 3/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1884941.8750 - mae: 781.0807 - mean_absolute_percentage_error: 4.4029
Epoch 4/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1824192.5000 - mae: 762.1395 - mean_absolute_percentage_error: 4.3021
Epoch 5/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2195694.5000 - mae: 849.5439 - mean_absolute_percentage_error: 4.9068
Epoch 6/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2768959.0000 - mae: 958.8622 - mean_absolute_percentage_error: 5.5885 
Epoch 7/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2575975.7500 - mae: 900.2877 - mean_absolute_percentage_error: 5.0858
Epoch 8/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1671730.0000 - mae: 695.0329 - mean_absolu

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 170739936.0000 - mae: 7825.7578 - mean_absolute_percentage_error: 48.4718   
Epoch 2/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4295304.5000 - mae: 1224.8947 - mean_absolute_percentage_error: 7.0994
Epoch 3/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3056317.0000 - mae: 1018.6688 - mean_absolute_percentage_error: 6.0647
Epoch 4/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2786894.0000 - mae: 972.0853 - mean_absolute_percentage_error: 5.6872
Epoch 5/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2244082.5000 - mae: 863.1904 - mean_absolute_percentage_error: 4.9565
Epoch 6/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1967227.7500 - mae: 805.3516 - mean_absolute_percentage_error: 4.6024
Epoch 7/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1721455.8750 - mae: 740.9312 - mean_absolute_percentage_error: 4.1861
Epoch 8/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1698521.7500 - mae: 735.5024 - mean_absol

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 114545944.0000 - mae: 6049.2017 - mean_absolute_percentage_error: 38.0497   
Epoch 2/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4352944.0000 - mae: 1269.5253 - mean_absolute_percentage_error: 7.6123
Epoch 3/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3134617.2500 - mae: 1028.3787 - mean_absolute_percentage_error: 5.9730
Epoch 4/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1550199.8750 - mae: 686.3116 - mean_absolute_percentage_error: 3.7831
Epoch 5/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2477985.0000 - mae: 902.4495 - mean_absolute_percentage_error: 5.2393
Epoch 6/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2186369.5000 - mae: 814.2457 - mean_absolute_percentage_error: 4.5454
Epoch 7/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1607988.1250 - mae: 682.6551 - mean_absolute_percentage_error: 3.7550
Epoch 8/250
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1922599.2500 - mae: 755.2423 - mean_absol

In [2]:
# Save the besto model for each period
# Bear Market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-REG-bear_market_1',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-REG-bear_market_1.keras')

# Bear Market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-REG-bear_market_2',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-REG-bear_market_2.keras')

# Bull Market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-REG-bull_market_1',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-REG-bull_market_1.keras')

# Bull Market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-REG-bull_market_2',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-REG-bull_market_2.keras')

Reloading Tuner from my_dir/CNN-3D-REG-bear_market_1/tuner0.json
Reloading Tuner from my_dir/CNN-3D-REG-bear_market_2/tuner0.json
Reloading Tuner from my_dir/CNN-3D-REG-bull_market_1/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Reloading Tuner from my_dir/CNN-3D-REG-bull_market_2/tuner0.json
